In [1]:
from terracatalogueclient import Catalogue
import datetime as dt
import glob
import pandas as pd
import geopandas as gpd
import sys, os

from util import suppress_stdout, create_gpd_for_scene

In [3]:
# input time frame in which reference scenes should be defined
# this should be no longer than 12 days! after 12 days, orbits of a single satellite repeat and ambiguities arise
start = dt.date(2021, 10, 1)
end = dt.date(2021, 10, 13)

aoi_file = "/home/jonathanbahlmann/Public/coherence-docs/aoi/belgium_france.geojson"
aoi = gpd.read_file(aoi_file)

ref_inter_bursts_file = "/home/jonathanbahlmann/Public/coherence-docs/src/intersecting_bursts.geojson"
#ref_inter_bursts = gpd.read_file(ref_inter_bursts_file)

print("starting search for scenes...")

catalogue = Catalogue()

cat = catalogue.get_products(
	"urn:eop:VITO:CGS_S1_SLC_L1",
	start = start,
	end = end
    # , geometry =  WKT string or shapely geom
)

s1a = []

for p in cat:
    path = p.data[0].href 
    iw_index = path.index("IW")
    vm_path = "/data/MTDA/CGS_S1/CGS_S1_SLC_L1/" + path[iw_index:]

    # make swath geometry and add basic info to df
    ana_split = create_gpd_for_scene(vm_path, make_regular = True)
    
    # append to list based on satellite
    if ana_split["sensor"].iloc[0] == "S1A":
        
        # AOI INTERSECTION
        # create boolean of which bursts intersect with aoi and which dont
        intersects = ana_split.intersects(aoi.iloc[0]["geometry"])
        # keep only those bursts that intersect with aoi
        intersecting_bursts = ana_split[intersects]
        
        if not intersecting_bursts.empty:
        
            # CHECK IF EXISTS
            rel_o = ana_split.iloc[0]["rel_orbit"]
            print(rel_o)
            o_dir = ana_split.iloc[0]["orbit_direction"]
            print(o_dir)
            #check = ref_inter_bursts.loc[(ref_inter_bursts["rel_orbit"] == rel_o) & (ref_inter_bursts["orbit_direction"] == o_dir)]
            # gpd.intersect(ana_split, check, how = "intersection").to_file("intersection_test_" + ana_split["id"] + ".geojson", driver = "GeoJSON")
            intersecting_bursts.to_file("b_test_" + ana_split.iloc[0]["id"] + ".geojson", driver = "GeoJSON")
            #check.to_file("c_test_" + ana_split.iloc[0]["id"] + ".geojson", driver = "GeoJSON")
        
            s1a.append(intersecting_bursts)
        
s1a_df = gpd.GeoDataFrame(pd.concat(s1a, ignore_index = True), crs=s1a[0].crs)

# create empty dictionairy for the mapping ID - frame number
relative_frames = {}
# initiate column
s1a_df["rel_frame"] = 0
# init frame number
fnr = int(1)

for index, row in s1a_df.iterrows():
    if row["id"] in relative_frames:
        row["rel_frame"] = relative_frames[row["id"]]
    else:
        row["rel_frame"] = fnr
        relative_frames[row["id"]] = fnr
        fnr += 1

# write bursts
s1a_df.to_file("reference_bursts.geojson", driver = "GeoJSON")
# extract scenes and write
s1a_df.dissolve(["id"], as_index = False).to_file("reference_scenes.geojson", driver = "GeoJSON")

# gpd.overlay(aoi, s1a_df, how = "intersection").to_file("test2.geojson", driver = "GeoJSON")

print("end")

starting search for scenes...
139
D
139
D
161
A
161
A
161
A
8
D
8
D
37
D
37
D
37
D
59
A
59
A
88
A
88
A
110
D
110
D
110
D
end


In [ ]:
dic = {"hi": 6}
print(dic["hi"])